In [2]:
if (!require(survRM2)) install.packages('survRM2')
library('survRM2') 
if (!require(mvtnorm)) install.packages('mvtnorm')
library('mvtnorm')
if (!require(survival)) install.packages('survival')
library('survival')
if (!require(nph)) install.packages('nph')
library('nph')
if (!require(foreach)) install.packages("foreach")
library('foreach') 
if (!require(doParallel)) install.packages("doParallel")
library('doParallel')
if (!require(ggplot2)) install.packages("ggplot2")
library('ggplot2')
if (!require(cowplot)) install.packages("cowplot")
library('cowplot')
library('IRdisplay')
if (!require(abind)) install.packages("abind")
library('abind')

n_cores <- detectCores()
cluster <- makeCluster(16) # How many cores we use
registerDoParallel(cluster)
# multi thread
invisible(clusterEvalQ(cluster, #import packages to parallel 
  {
  library('survRM2')
  library('mvtnorm')
  library('survival')
  library('nph')
  library("simtrial")
  library("foreach")
  }))

source('/home/r27user6/RMST_Code/Function.R')
clusterExport(cluster, "expo_gen_2stages")

# 3 Methods Comparison Under Proportional Hazard

### 2 stages Rejection region of overall:
### 1. Log rank test
#### $\textcolor{lightgreen}{\frac{W_1}{\sigma_1}>c_1\ \&\ \frac{W}{\sigma}>c}$
### 2. Simple RMST Difference
#### $ \textcolor{lightgreen}{E(\tau_1) - C(\tau_1) > m_1\ \&\  E(\tau_2)-C(\tau_2)>m_2 }$
### 3. Our RMST Rejection method
#### $ \textcolor{lightgreen}{E(\tau_1)-C(\tau_1)>m_1\ \&\  E(\tau_1)>t_1\ \&\  E(\tau_2)-C(\tau_2)>m_2\ \&\  E(\tau_2)>t_2}$
--------

#### Different from the setting in Jung's 2017 table 1:
#### We set the total accrual time = 2 years and follow up 1 year
#### Recruit rate is 60(30 per arm) per year calculate the test power of 3 method controling $\alpha = 0.05$
#### Interim period we set at $c(1,\ 1.2,\ 1.4,\ 1.6,\ 1.8)$ year


In [14]:
sim_size <- 5000
acc_time <- 2
cen_time <- 1
delta <- 1.5
lambda_H1 <- 0.9
lambda_H0 <- 0.9 * delta
r <- 60  # patient recruit rate
acc_time <- 2
interim_list <- c(1, 1.2, 1.4, 1.6, 1.8)
n <- acc_time * r / 2  # total sample size per arm

data_C <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, dist = 'exp', 
                            cen_time = cen_time, arm = 0, interim = interim_list)    
data_E_H0 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, dist = 'exp', 
                            cen_time = cen_time, arm = 1, interim = interim_list)                    
data_E_H1 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H1, dist = 'exp', 
                            cen_time = cen_time, arm = 1, interim = interim_list)

test_data <- foreach(i = 1 : length(interim_list), .combine = 'cbind', .packages = 'foreach') %dopar% 
    {
        interim <- interim_list[i]
        rmst_h0_int <- RMST_sim_cal(n = n,data_E = data_E_H0[ , c(2,3,1), i], 
                                data_C = data_C[ , c(2,3,1), i],tau = interim, sim_size = sim_size)
        rmst_h1_int <- RMST_sim_cal(n = n,data_E = data_E_H1[ , c(2,3,1), i], 
                                data_C = data_C[ , c(2,3,1), i],tau = interim, sim_size = sim_size)
        rmst_h0_fin <- RMST_sim_cal(n = n,data_E = data_E_H0[ , c(4,5,1), i], 
                                data_C = data_C[ , c(4,5,1), i],tau = acc_time + cen_time,sim_size = sim_size)
        rmst_h1_fin <- RMST_sim_cal(n = n,data_E = data_E_H1[ , c(4,5,1), i], 
                                data_C = data_C[ , c(4,5,1), i],tau = acc_time + cen_time,sim_size = sim_size)
        rmst_data <- rbind(rmst_h0_int, rmst_h1_int, rmst_h0_fin, rmst_h1_fin)
        #Log rank
        z_stats_h0_int <- log_rank_sim(data_C = data_C[ , c(2,3,1), i], 
                                data_E = data_E_H0[ , c(2,3,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        z_stats_h1_int <- log_rank_sim(data_C = data_C[ , c(2,3,1), i], 
                                data_E = data_E_H1[ , c(2,3,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        z_stats_h0_fin <- log_rank_sim(data_C = data_C[ , c(4,5,1), i], 
                                data_E = data_E_H0[ , c(4,5,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        z_stats_h1_fin <- log_rank_sim(data_C = data_C[ , c(4,5,1), i], 
                                data_E = data_E_H1[ , c(4,5,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        logrank_data <- rbind(z_stats_h0_int, z_stats_h1_int, z_stats_h0_fin, z_stats_h1_fin)
        return(list(rmst_data = rmst_data, logrank_data = logrank_data))
    }


In [28]:
test_data[,1]$logrank_data

,result.1,result.2,result.3,result.4,result.5,result.6,result.7,result.8,result.9,result.10,⋯,result.4991,result.4992,result.4993,result.4994,result.4995,result.4996,result.4997,result.4998,result.4999,result.5000
z_stats_h0_int,-0.06362659,-1.9482975,0.4941906,0.9826726,-1.0889203,-1.6114202,-0.8804308,0.91203955,-0.002453364,-0.7166760,⋯,0.4651835,-0.76867322,1.6936863,-0.3594900,0.005801764,0.6119510,-0.4710424,0.03883499,-0.4701056,-0.8274910
z_stats_h1_int,0.25862594,1.8046191,0.2523339,3.3720426,0.6177109,-1.0570513,0.5133817,2.62933285,0.473871271,0.2233641,⋯,1.6134682,-0.22099132,0.7669526,0.8400872,1.928472005,2.6077548,0.8139987,1.23482100,1.4902955,1.7672285
z_stats_h0_fin,0.17703856,0.6174016,0.3393320,1.5354990,-1.3942624,-2.6290006,-1.3772224,0.01216219,-0.035932570,-0.8888801,⋯,-0.9475598,-0.03573187,0.3269426,-1.1470394,-1.127876730,0.3715104,0.1929692,0.86966320,1.6171024,-0.6185291
z_stats_h1_fin,1.20905441,2.5118186,2.2023461,3.8111299,2.2928636,0.2557972,2.3085942,3.37511838,0.647333648,0.6734068,⋯,0.9712828,1.76799959,1.2345521,1.4560690,1.921450974,3.0721935,1.6451877,2.56324443,3.6778658,3.3925848


In [19]:
dim(test_data)

[1] 2 5

## <span style="color:yellow">Grid search remarks:
### The searching range should be consistant under a specific experiment setting
###  1. Searching range of m (RMST difference):
#### $\qquad$ Lower bound slightly smaller than $E_1 - C_1|H_0\ $; Upper bound larger than $E_2 - C_2|H_1$
#### $\qquad$ upper bound is controled by m_low + search_times * search_step
### 2. Searching range of t (RMST of experiment group):
#### $\qquad$ t_low slightly smaller than $E_1\ $; t_up slightly larger than $E_2$
#### $\qquad$ search_step of t = (upper - lower) / searching times
### 3. Searching range of c (critical value of Log rank):
#### $\qquad$ Lower bound slightly smaller than $\frac{W_1}{\sigma_1}|H_0\ $; Upper bound larger than $\frac{W}{\sigma}|H_1$
### · <font color=LightSeaGreen>Control PET0, PET1 and overall $\alpha$, search for the highest power

In [26]:
# search for parameter under different interim period
best_result <- foreach(i = 1 : length(interim_list), .combine = 'cbind', .packages = 'foreach') %dopar% 
    {   rmst_data <- test_data[,i]$rmst_data
        logrank_data <- test_data[,i]$logrank_data
        search_times <- 500

        #RMST
        m_low <- mean(rmst_data[2,] - rmst_data[1,]) - 3 * sd(rmst_data[2,] - rmst_data[1,]) 
        # Smallest RMST difference: interim under H0
        m_up <- mean(rmst_data[8,] - rmst_data[7,]) + 3 * sd(rmst_data[8,] - rmst_data[7,]) 
        # Largest RMST difference: final under H1
        search_step_rmst <- (m_up - m_low) / search_times
        t_low <- mean(rmst_data[2,]) - 3 * sd(rmst_data[2,])
        # Smallest experiment RMST: interim under H0
        t_up <- mean(rmst_data[8,]) + 3 * sd(rmst_data[8,]) 

        #Log rank
        c_low <- mean(logrank_data[1,]) - 3 * sd(logrank_data[1,])
        c_up <- mean(logrank_data[4,]) + 3 * sd(logrank_data[4,])
        search_step_lr <- (c_up - c_low) / search_times

        # Control PET0 and PET1
        tar_a1 <- 0.8      #Upper bound of 1-PET0
        tar_pow1_low <- 0.2  # lower bound of 1-PET1
        tar_alpha <- 0.05

        best_our_rmst <- find_m_t_RMST(m_low = m_low, t_low = t_low, t_up = t_up, rmst_data = rmst_data, 
                            search_times = search_times, search_step = search_step_rmst, tar_a1 = tar_a1, 
                            tar_pow1_low = tar_pow1_low, tar_a2 = tar_alpha, sim_size = sim_size)

        best_simple_rmst <- find_m_t_RMST(m_low = m_low, t_low = -Inf, t_up = t_up, rmst_data = rmst_data, 
                            search_times = search_times, search_step = search_step_rmst, tar_a1 = tar_a1, 
                            tar_pow1_low = tar_pow1_low, tar_a2 = tar_alpha, sim_size = sim_size)

        best_log_rank <- find_m_logrank(m_low = c_low, logrank_data = logrank_data, 
                            search_times = search_times, search_step = search_step_lr, tar_a1 = tar_a1, 
                            tar_pow1_low = tar_pow1_low, tar_a2 = tar_alpha, sim_size = sim_size)
        # best result in dataframe
        return(list(best_our_rmst = best_our_rmst, 
                    best_simple_rmst = best_simple_rmst,
                    best_log_rank = best_log_rank))
    }

In [30]:
dim(best_result)

[1] 3 5

In [31]:
best_result[3,]

m1,PET0,PET1,m2,alpha,Power
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.870153,0.2038,0.0112,1.660033,0.0482,0.6502
